In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
traindata_standard=joblib.load('traindata_standard_03_07_2019__10_07_2019.pkl')

# ЗАПИСЬ ИЗ ДАТАФРЕЙМА В БАЗУ ДАННЫХ НА ЖЕСТКОМ ДИСКЕ

In [2]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)


class ITEMS(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    Date = db.Column(db.DateTime, nullable=False)
    High = db.Column(db.FLOAT, unique=False, nullable=False)
    Low = db.Column(db.FLOAT, unique=False, nullable=False)
    company = db.Column(db.String(80), unique=False, nullable=False)
    def __repr__(self):
        #return '<company %r>' % self.company
        #return (self.Date, self.High, self.Low, self.company)
        return "(%r, %r, %r, %r)" % (self.Date, self.High, self.Low, self.company)
    def to_dict(self):
        return {'Date':self.Date, 'High':self.High, 'Low':self.Low,
               'company':self.company}

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [4]:
db.create_all()

In [7]:
#for firm in list(traindata_standard.keys()):
#    df = traindata_standard[firm]
#    for i in range(len(df)):
#        item = ITEMS(Date=df.index[i], High=df['High'][i],
 #                  Low=df['Low'][i], company=df['company'][i])
 #       db.session.add(item)
#db.session.commit()

In [8]:
objects = []
for firm in list(traindata_standard.keys()):
    df = traindata_standard[firm]
    for i in range(len(df)):
        item = ITEMS(Date=df.index[i], High=df['High'][i],
                   Low=df['Low'][i], company=df['company'][i])
        objects.append(item)

In [11]:
db.session.bulk_save_objects(objects)
db.session.commit()

# ВОССТАНОВЛЕНИЕ ИЗ БАЗЫ ДАННЫХ ОБРАТНО В ДАТАФРЕЙМ

In [3]:
item = ITEMS.query.filter_by(company='FBHS').all()

In [15]:
example = pd.DataFrame([i.to_dict() for i in item])
example.index = example['Date']
example.drop('Date',axis=1,inplace=True)

In [16]:
example.head()

,High,Low,company
Date,,,
2019-07-03 09:33:00,56.530,56.530,FBHS
2019-07-03 09:34:00,56.390,56.390,FBHS
2019-07-03 09:38:00,56.480,56.480,FBHS
2019-07-03 09:40:00,56.430,56.430,FBHS
2019-07-03 09:41:00,56.375,56.375,FBHS
